<a href="https://colab.research.google.com/github/Saputoa21/Machine-Translation/blob/main/filterMT_AdvancedMT_2025W.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#https://github.com/ymoslem/Adaptive-MT-LLM-Fine-tuning/blob/main/Data-Processing-Adaptive-MT.ipynb
#https://github.com/ymoslem/MT-Preparation/tree/main

In [2]:
!git clone https://github.com/ymoslem/MT-Preparation.git

Cloning into 'MT-Preparation'...
remote: Enumerating objects: 323, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 323 (delta 35), reused 21 (delta 20), pack-reused 268 (from 2)
Receiving objects: 100% (323/323), 94.95 KiB | 1.48 MiB/s, done.
Resolving deltas: 100% (156/156), done.


In [3]:
%cd MT-Preparation

/content/MT-Preparation


In [4]:
!pip3 install --user -r requirements.txt

In [5]:
%cd ..

/content


In [7]:
!ls

MT-Preparation		    train.en-de.de-filtered.de.semantic.de
sample_data		    train.en-de.en
train.en-de.de		    train.en-de.en-filtered.en
train.en-de.de-filtered.de  train.en-de.en-filtered.en.semantic.en


In [8]:
!python3 MT-Preparation/filtering/filter.py train.en-de.en train.en-de.de en de

Dataframe shape (rows, columns): (50000, 2)
--- Rows with Empty Cells Deleted	--> Rows: 49997
--- Duplicates Deleted			--> Rows: 49997
--- Source-Copied Rows Deleted		--> Rows: 49997
--- Too Long Source/Target Deleted	--> Rows: 48399
--- Too Short Source/Target Deleted	--> Rows: 48321
--- HTML Removed			--> Rows: 48321
--- Rows will remain true-cased		--> Rows: 48321
--- Rows with Empty Cells Deleted	--> Rows: 48315
--- Rows Shuffled			--> Rows: 48315
--- Source Saved: train.en-de.en-filtered.en
--- Target Saved: train.en-de.de-filtered.de


In [9]:
!python3 MT-Preparation/filtering/semantic_filter.py train.en-de.en-filtered.en train.en-de.de-filtered.de en de 10000 0.45

2025-10-26 12:16:06.232973: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761480966.256444    7649 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761480966.267883    7649 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761480966.292401    7649 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761480966.292437    7649 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761480966.292445    7649 computation_placer.cc:177] computation placer alr

In [10]:
!wc -l *

wc: model_cache: Is a directory
       0 model_cache
wc: MT-Preparation: Is a directory
       0 MT-Preparation
wc: sample_data: Is a directory
       0 sample_data
   50000 train.en-de.de
   48315 train.en-de.de-filtered.de
   47614 train.en-de.de-filtered.de.semantic.de
   50000 train.en-de.en
   48315 train.en-de.en-filtered.en
   47614 train.en-de.en-filtered.en.semantic.en
  291858 total


# **My corpus text**


I have chosen MultiUN (v1) corpus with the language pair EN-RU and size of ... sentences. I splitted it into train () and test () sets.

The files are:

*   UN-train.en-ru.en
*   UN-train.en-ru.ru
*   UN-test.en-ru.en
*   UN-test.en-ru.ru

In [ ]:
!git clone https://github.com/ymoslem/MT-Preparation.git

Cloning into 'MT-Preparation'...
remote: Enumerating objects: 323, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 323 (delta 35), reused 21 (delta 20), pack-reused 268 (from 2)
Receiving objects: 100% (323/323), 94.95 KiB | 1.48 MiB/s, done.
Resolving deltas: 100% (156/156), done.


In [ ]:
%cd MT-Preparation

/content/MT-Preparation


In [ ]:
!pip3 install --user -r requirements.txt

In [ ]:
%cd ..

/content


In [ ]:
!ls

MT-Preparation		    train.en-de.de-filtered.de.semantic.de
sample_data		    train.en-de.en
train.en-de.de		    train.en-de.en-filtered.en
train.en-de.de-filtered.de  train.en-de.en-filtered.en.semantic.en


In [ ]:
!python3 MT-Preparation/filtering/filter.py train.en-de.en train.en-de.de en de

Dataframe shape (rows, columns): (50000, 2)
--- Rows with Empty Cells Deleted	--> Rows: 49997
--- Duplicates Deleted			--> Rows: 49997
--- Source-Copied Rows Deleted		--> Rows: 49997
--- Too Long Source/Target Deleted	--> Rows: 48399
--- Too Short Source/Target Deleted	--> Rows: 48321
--- HTML Removed			--> Rows: 48321
--- Rows will remain true-cased		--> Rows: 48321
--- Rows with Empty Cells Deleted	--> Rows: 48315
--- Rows Shuffled			--> Rows: 48315
--- Source Saved: train.en-de.en-filtered.en
--- Target Saved: train.en-de.de-filtered.de
